In [1]:
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import roc_auc_score,roc_curve

from catboost import CatBoostClassifier

from data_manager import DataManager
from tqdm import tqdm_notebook as tqdm
import pickle

In [2]:
def plot_roc_auc(model, X_train, X_val, y_train, y_val):
    y_train_pred = model.predict_proba(X_train)[:,1]
    print("Train ROC AUC:",roc_auc_score(y_train,y_train_pred))

    fpr,tpr,_ = roc_curve(y_train, y_train_pred)
    plt.plot(fpr,tpr,label='train AUC')

    y_val_pred = model.predict_proba(X_val)[:,1]
    print("Val ROC AUC:",roc_auc_score(y_val,y_val_pred))

    fpr,tpr,_ = roc_curve(y_val, y_val_pred)
    plt.plot(fpr,tpr,label='validation AUC')

    plt.plot([0,1],[0,1])
    plt.legend(loc='lower right')
    
def plot_feature_importances(model, X):
    order = np.argsort(model._feature_importance)
    plt.figure(figsize=[6,9])
    plt.plot(np.array(model._feature_importance)[order],range(len(order)),marker='o')
    plt.hlines(range(len(order)),np.zeros_like(order),np.array(model._feature_importance)[order],linestyles=':')
    plt.yticks(range(X.shape[1]),X.columns[order]);
    plt.tick_params(labelsize=16)
    plt.xlim([0.1,max(model._feature_importance)*1.5])
    plt.ylim(-1,len(order))
    plt.xscale('log') 

In [ ]:
dm = DataManager(city_name='spb')

In [ ]:
with open('dm_spb.pickle', 'wb') as f:
    pickle.dump(dm, f)

In [3]:
with open('dm_spb.pickle', 'rb') as f:
    dm = pickle.load(f)

In [4]:
X, y, block_ids = dm.X_train, dm.y_train, dm.train_block_ids

In [6]:
X = X.fillna(-999)

In [7]:
in_train = block_ids['hours_since'] <= np.percentile(block_ids['hours_since'], 85)  #leave last 15% for validation
X_train, y_train = X[in_train], y[in_train]
X_val, y_val = X[~in_train], y[~in_train]

In [8]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((28041, 73), (4762, 73), (28041,), (4762,))

In [9]:
model = CatBoostClassifier(iterations=2000,
                           depth=6,
                           loss_function='Logloss',
                           learning_rate=0.015,
                           thread_count=12,
                           use_best_model=True,
                           eval_metric='AUC',
                           random_seed=1,
                           verbose=True) \
            .fit(X_train, y_train, eval_set=(X_val, y_val))

Borders for float features generated
0:	learn 0.5759663286	test 0.5399139823	bestTest 0.5399139823		total: 88.2ms	remaining: 2m 56s
1:	learn 0.5865843208	test 0.5290183933	bestTest 0.5399139823		total: 127ms	remaining: 2m 6s
2:	learn 0.6072525222	test 0.5343802783	bestTest 0.5399139823		total: 162ms	remaining: 1m 48s
3:	learn 0.6136458818	test 0.5466863824	bestTest 0.5466863824		total: 200ms	remaining: 1m 39s
4:	learn 0.6353058657	test 0.5679106277	bestTest 0.5679106277		total: 233ms	remaining: 1m 33s
5:	learn 0.634475304	test 0.5677698348	bestTest 0.5679106277		total: 265ms	remaining: 1m 28s
6:	learn 0.66282094	test 0.5475562902	bestTest 0.5679106277		total: 299ms	remaining: 1m 25s
7:	learn 0.6626440435	test 0.5509301502	bestTest 0.5679106277		total: 330ms	remaining: 1m 22s
8:	learn 0.6647130079	test 0.5454126644	bestTest 0.5679106277		total: 361ms	remaining: 1m 19s
9:	learn 0.6643980952	test 0.5482475624	bestTest 0.5679106277		total: 392ms	remaining: 1m 17s
10:	learn 0.6644163031	tes

93:	learn 0.6946067426	test 0.553164504	bestTest 0.5679106277		total: 2.89s	remaining: 58.7s
94:	learn 0.6951150576	test 0.5540713141	bestTest 0.5679106277		total: 2.93s	remaining: 58.7s
95:	learn 0.694714991	test 0.5533624133	bestTest 0.5679106277		total: 2.96s	remaining: 58.7s
96:	learn 0.6949094349	test 0.5536599824	bestTest 0.5679106277		total: 3s	remaining: 58.8s
97:	learn 0.6951992545	test 0.5532763862	bestTest 0.5679106277		total: 3.03s	remaining: 58.8s
98:	learn 0.6979730951	test 0.5580158675	bestTest 0.5679106277		total: 3.06s	remaining: 58.7s
99:	learn 0.7001493932	test 0.5571367929	bestTest 0.5679106277		total: 3.09s	remaining: 58.7s
100:	learn 0.7007649572	test 0.5584243787	bestTest 0.5679106277		total: 3.12s	remaining: 58.7s
101:	learn 0.7017904286	test 0.5587478029	bestTest 0.5679106277		total: 3.15s	remaining: 58.6s
102:	learn 0.7019691714	test 0.5591516131	bestTest 0.5679106277		total: 3.18s	remaining: 58.5s
103:	learn 0.7022502173	test 0.5588465225	bestTest 0.567910627

184:	learn 0.7405190982	test 0.6074391335	bestTest 0.6074391335		total: 5.6s	remaining: 54.9s
185:	learn 0.7400310914	test 0.607048956	bestTest 0.6074391335		total: 5.63s	remaining: 54.9s
186:	learn 0.7413527171	test 0.6093627555	bestTest 0.6093627555		total: 5.66s	remaining: 54.8s
187:	learn 0.7416390984	test 0.6094981424	bestTest 0.6094981424		total: 5.69s	remaining: 54.8s
188:	learn 0.7415081537	test 0.6089706977	bestTest 0.6094981424		total: 5.72s	remaining: 54.8s
189:	learn 0.7420966344	test 0.6102155049	bestTest 0.6102155049		total: 5.75s	remaining: 54.8s
190:	learn 0.7424537811	test 0.6109991505	bestTest 0.6109991505		total: 5.78s	remaining: 54.7s
191:	learn 0.7429038646	test 0.6116065112	bestTest 0.6116065112		total: 5.8s	remaining: 54.7s
192:	learn 0.7436620264	test 0.6113662935	bestTest 0.6116065112		total: 5.83s	remaining: 54.6s
193:	learn 0.7441146166	test 0.611545399	bestTest 0.6116065112		total: 5.86s	remaining: 54.6s
194:	learn 0.7446588055	test 0.6119266448	bestTest 0.6

275:	learn 0.7765574804	test 0.6535999515	bestTest 0.6535999515		total: 8.3s	remaining: 51.9s
276:	learn 0.7765640014	test 0.653401102	bestTest 0.6535999515		total: 8.33s	remaining: 51.8s
277:	learn 0.7772640164	test 0.6546788158	bestTest 0.6546788158		total: 8.36s	remaining: 51.8s
278:	learn 0.7777470435	test 0.655113182	bestTest 0.655113182		total: 8.39s	remaining: 51.8s
279:	learn 0.7781115412	test 0.6552847661	bestTest 0.6552847661		total: 8.42s	remaining: 51.7s
280:	learn 0.778560964	test 0.6557957576	bestTest 0.6557957576		total: 8.45s	remaining: 51.7s
281:	learn 0.7784969229	test 0.6556302847	bestTest 0.6557957576		total: 8.48s	remaining: 51.7s
282:	learn 0.7788598792	test 0.6570654798	bestTest 0.6570654798		total: 8.51s	remaining: 51.7s
283:	learn 0.7794662461	test 0.6578260908	bestTest 0.6578260908		total: 8.54s	remaining: 51.6s
284:	learn 0.7795725466	test 0.6578768609	bestTest 0.6578768609		total: 8.57s	remaining: 51.6s
285:	learn 0.7798833689	test 0.657877331	bestTest 0.657

366:	learn 0.7981827252	test 0.6697763343	bestTest 0.6697763343		total: 11s	remaining: 48.7s
367:	learn 0.7982982906	test 0.6699112511	bestTest 0.6699112511		total: 11s	remaining: 48.7s
368:	learn 0.7985108747	test 0.6699596707	bestTest 0.6699596707		total: 11s	remaining: 48.7s
369:	learn 0.7989605009	test 0.6703559594	bestTest 0.6703559594		total: 11s	remaining: 48.6s
370:	learn 0.7992736436	test 0.6711602891	bestTest 0.6711602891		total: 11.1s	remaining: 48.6s
371:	learn 0.7998623784	test 0.6708650705	bestTest 0.6711602891		total: 11.1s	remaining: 48.6s
372:	learn 0.7999193905	test 0.6709398154	bestTest 0.6711602891		total: 11.1s	remaining: 48.5s
373:	learn 0.8002016221	test 0.6710140901	bestTest 0.6711602891		total: 11.2s	remaining: 48.5s
374:	learn 0.8001661039	test 0.6706887855	bestTest 0.6711602891		total: 11.2s	remaining: 48.5s
375:	learn 0.8003715742	test 0.6705891257	bestTest 0.6711602891		total: 11.2s	remaining: 48.4s
376:	learn 0.8005321767	test 0.670706179	bestTest 0.671160

...

1636:	learn 0.9210246096	test 0.7166197278	bestTest 0.7173037137		total: 48.3s	remaining: 10.7s
1637:	learn 0.9210453921	test 0.7165919923	bestTest 0.7173037137		total: 48.4s	remaining: 10.7s
1638:	learn 0.9210732206	test 0.7165778895	bestTest 0.7173037137		total: 48.4s	remaining: 10.7s
1639:	learn 0.9210999312	test 0.7165708381	bestTest 0.7173037137		total: 48.4s	remaining: 10.6s
1640:	learn 0.9211446296	test 0.7165355811	bestTest 0.7173037137		total: 48.5s	remaining: 10.6s
1641:	learn 0.9211734236	test 0.7165412223	bestTest 0.7173037137		total: 48.5s	remaining: 10.6s
1642:	learn 0.9212233726	test 0.7165515643	bestTest 0.7173037137		total: 48.5s	remaining: 10.5s
1643:	learn 0.9212753711	test 0.716489512	bestTest 0.7173037137		total: 48.5s	remaining: 10.5s
1644:	learn 0.9213377524	test 0.7164335709	bestTest 0.7173037137		total: 48.6s	remaining: 10.5s
1645:	learn 0.9213613465	test 0.716499854	bestTest 0.7173037137		total: 48.6s	remaining: 10.5s
1646:	learn 0.9213557909	test 0.7164810503

1727:	learn 0.9250509365	test 0.7147149095	bestTest 0.7173037137		total: 51s	remaining: 8.03s
1728:	learn 0.9250949405	test 0.7147003366	bestTest 0.7173037137		total: 51.1s	remaining: 8s
1729:	learn 0.9251394187	test 0.7147478161	bestTest 0.7173037137		total: 51.1s	remaining: 7.97s
1730:	learn 0.9252489881	test 0.7147882441	bestTest 0.7173037137		total: 51.1s	remaining: 7.94s
1731:	learn 0.9252801872	test 0.7147670899	bestTest 0.7173037137		total: 51.1s	remaining: 7.91s
1732:	learn 0.9252996316	test 0.7148564076	bestTest 0.7173037137		total: 51.2s	remaining: 7.88s
1733:	learn 0.9253088287	test 0.7148385441	bestTest 0.7173037137		total: 51.2s	remaining: 7.85s
1734:	learn 0.9253338456	test 0.7147243114	bestTest 0.7173037137		total: 51.2s	remaining: 7.82s
1735:	learn 0.9253685508	test 0.714823031	bestTest 0.7173037137		total: 51.3s	remaining: 7.79s
1736:	learn 0.9254334388	test 0.714692345	bestTest 0.7173037137		total: 51.3s	remaining: 7.76s
1737:	learn 0.9254614706	test 0.7146777722	best

1817:	learn 0.9284501987	test 0.713695277	bestTest 0.7173037137		total: 53.7s	remaining: 5.37s
1818:	learn 0.9285268246	test 0.713725363	bestTest 0.7173037137		total: 53.7s	remaining: 5.34s
1819:	learn 0.9285737587	test 0.7137855349	bestTest 0.7173037137		total: 53.7s	remaining: 5.31s
1820:	learn 0.9286174408	test 0.7138207919	bestTest 0.7173037137		total: 53.8s	remaining: 5.28s
1821:	learn 0.9286506555	test 0.7137446368	bestTest 0.7173037137		total: 53.8s	remaining: 5.25s
1822:	learn 0.9286819224	test 0.7137596798	bestTest 0.7173037137		total: 53.8s	remaining: 5.23s
1823:	learn 0.9286944731	test 0.7137625004	bestTest 0.7173037137		total: 53.9s	remaining: 5.2s
1824:	learn 0.9287810075	test 0.7137775434	bestTest 0.7173037137		total: 53.9s	remaining: 5.17s
1825:	learn 0.9287985548	test 0.7137925863	bestTest 0.7173037137		total: 53.9s	remaining: 5.14s
1826:	learn 0.9288320574	test 0.7138494676	bestTest 0.7173037137		total: 53.9s	remaining: 5.11s
1827:	learn 0.9288307024	test 0.7138071592	

1908:	learn 0.9318661954	test 0.7123602119	bestTest 0.7173037137		total: 56.4s	remaining: 2.69s
1909:	learn 0.9318805754	test 0.7123644427	bestTest 0.7173037137		total: 56.4s	remaining: 2.66s
1910:	learn 0.9319015442	test 0.7123475193	bestTest 0.7173037137		total: 56.4s	remaining: 2.63s
1911:	learn 0.9319405176	test 0.7124044006	bestTest 0.7173037137		total: 56.5s	remaining: 2.6s
1912:	learn 0.9319891456	test 0.7122798259	bestTest 0.7173037137		total: 56.5s	remaining: 2.57s
1913:	learn 0.9320371807	test 0.7123465792	bestTest 0.7173037137		total: 56.5s	remaining: 2.54s
1914:	learn 0.9320606393	test 0.7123094418	bestTest 0.7173037137		total: 56.5s	remaining: 2.51s
1915:	learn 0.9320783391	test 0.712360682	bestTest 0.7173037137		total: 56.6s	remaining: 2.48s
1916:	learn 0.9320872144	test 0.712386067	bestTest 0.7173037137		total: 56.6s	remaining: 2.45s
1917:	learn 0.9320985287	test 0.712391238	bestTest 0.7173037137		total: 56.6s	remaining: 2.42s
1918:	learn 0.9321370449	test 0.7123592717	b

1999:	learn 0.9344536204	test 0.7121199942	bestTest 0.7173037137		total: 59s	remaining: 0us

bestTest = 0.7173037137
bestIteration = 1221

Shrink model to first 1222 iterations.


In [ ]:
tree_count = model.tree_count_
print(tree_count)

In [ ]:
def stas_xgb(X_train, y_train, X_val, y_val=None):
    ytestxgb = np.zeros(X_val.shape[0])
    bgs = 10
    for bg in tqdm(range(bgs)):
        seed = bg + 1

        model = CatBoostClassifier(iterations=tree_count,
                                   depth=6,
                                   loss_function='Logloss',
                                   learning_rate=0.015,
                                   thread_count=12,
                                   eval_metric='AUC',
                                   random_seed=seed) \
                    .fit(X_train, y_train)
        
        ypredxgb = model.predict_proba(X_val)[:, 1]
        ytestxgb += ypredxgb
        
        if y_val is not None:
            print(bg, roc_auc_score(y_val, ytestxgb / (bg + 1.)), roc_auc_score(y_val, ypredxgb))
        else:
            print(bg)
    
    ytestxgb /= bgs
    return ytestxgb    

In [ ]:
stas_xgb(X_train, y_train, X_val, y_val)

In [ ]:
model.predict

In [ ]:
plot_roc_auc(model, dm.X_train, dm.X_val, dm.y_train, dm.y_val)

In [ ]:
plot_feature_importances(model, dm.X_train)

## Final model and uploading the results

In [ ]:
y_pred = stas_xgb(X, y, X_test)

In [ ]:
X_test, test_block_ids = dm.X_test, dm.test_block_ids

In [ ]:
#This code saves the prediction for one city.
prediction_for_one_city = test_block_ids.copy()
prediction_for_one_city["prediction"] = model.predict_proba(X_test)[:,1]
prediction_for_one_city.to_csv(CITY_PREDICTIONS_PATH)

prediction_for_one_city.head()

#WARNING! you must run this notebook for all three regions before proceeding!
#We assume that you have prediction_msk.csv , prediction_spb.csv and prediction_kazan.csv files prepared.

In [ ]:
data = X.copy()
data["target"] = y
data.to_csv("intermediate_data/spb.csv")
X_test.to_csv("intermediate_data/spb_test.csv")

In [ ]:
predictions = pd.concat(
    [pd.read_csv(fname,index_col=0) for fname in ("./intermediate_data/prediction_kazan.csv",
                                                  "./intermediate_data/prediction_spb.csv",
                                                  "./intermediate_data/prediction_msk.csv")],
    ignore_index=True
)
blocks = pd.read_csv("./data/hackathon_tosubmit.tsv",sep='\t')
assert len(predictions) == len(blocks),"Predictions don't match blocks. Sumbit at your own risk."

merged = pd.merge(blocks,predictions,how='left',on=["sq_x","sq_y","hour_hash"])
assert not np.isnan(merged.prediction).any(), "some predictions are missing. Sumbit at your own risk."

In [ ]:
merged[['id','prediction']].to_csv("baseline_submission.csv",sep=',',index=False,header=False)

In [ ]:
!head baseline_submission.csv